<a href="https://colab.research.google.com/github/santosh3110/Trading_Strategies/blob/main/rsi%20strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# pip install vectorbt

In [39]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from datetime import datetime

In [40]:
btc_price = pd.read_csv("/content/btc_17-18_15m.csv")[["datetime","close"]]
# btc_price["date"]=pd.to_datetime(btc_price["timestamp"], unit="s")
btc_price = btc_price.set_index("datetime")["close"]

num = 10
metric = "total_return"
btc_price

,close
datetime,
2017-08-17 09:30:00,4261.48
2017-08-17 09:45:00,4261.45
2017-08-17 10:00:00,4310.07
2017-08-17 10:15:00,4308.83
2017-08-17 10:30:00,4304.31
...,...
2018-01-01 04:30:00,13733.71
2018-01-01 04:45:00,13724.00
2018-01-01 05:00:00,13755.27


In [41]:
rsi = vbt.RSI.run(btc_price, window = 14, short_name="rsi")
num = 10
entry_points = np.linspace(45,55, num=num)
exit_points = np.linspace(70,90, num=num)
grid = np.array(np.meshgrid(entry_points, exit_points)).T.reshape(-1,2)
entries = rsi.rsi_crossed_below(list(grid[:, [0]]))
exits = rsi.rsi_crossed_above(list(grid[:, [1]]))
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, freq="1h")

pf_perf = pf.deep_getattr(metric)

pf_perf_matrix = pf_perf.vbt.unstack_to_df(
        index_levels = "rsi_crossed_above",
        column_levels = "rsi_crossed_below")

pf_perf_matrix.vbt.heatmap(
        xaxis_title = "entry",
        yaxis_title="exit").show()

In [42]:
# # Assuming btc_price is already defined
# # Calculate RSI and its moving average
# rsi = vbt.RSI.run(btc_price, window=14, short_name="rsi")
# rsi_sma = vbt.MA.run(rsi.rsi, window=20)

# # Buy and Sell Entry Signals
# buy_entries = rsi_sma.ma_crossed_above(50)  # RSI SMA crosses above 50
# sell_entries = rsi_sma.ma_crossed_below(50)  # RSI SMA crosses below 50

# # Define entry/exit threshold grids
# entry_thresholds = [50]  # Fixed at 50 for simplicity
# exit_thresholds = np.linspace(70, 80, num=11)  # Exit thresholds for buy
# sell_exit_thresholds = np.linspace(30, 20, num=11)  # Exit thresholds for sell

# # Create a grid of portfolios for analysis
# performance_matrix = []

# for buy_exit in exit_thresholds:
#     row_performance = []
#     for sell_exit in sell_exit_thresholds:
#         # Generate exit signals for the current thresholds
#         buy_exits = rsi.rsi_crossed_above(buy_exit)
#         sell_exits = rsi.rsi_crossed_below(sell_exit)

#         # Combine signals into a portfolio simulation
#         pf = vbt.Portfolio.from_signals(
#             btc_price,
#             entries=buy_entries | sell_entries,  # Combine buy and sell entries
#             exits=buy_exits | sell_exits,        # Combine buy and sell exits
#             freq="15m"
#         )

#         # Extract performance metric
#         row_performance.append(pf.total_return())
#     performance_matrix.append(row_performance)

# # Convert performance_matrix to a DataFrame
# performance_df = pd.DataFrame(
#     performance_matrix,
#     index=[f"Buy Exit: {x}" for x in exit_thresholds],
#     columns=[f"Sell Exit: {x}" for x in sell_exit_thresholds]
# )

# # Plot a heatmap
# performance_df.vbt.heatmap(
#     xaxis_title="Sell Exit Threshold",
#     yaxis_title="Buy Exit Threshold"
# ).show()


In [43]:
pf.stats()

<ipython-input-43-9995df72936d>:1: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x79be5f761240>. Pass column to select a single column/group.



,agg_func_mean
Start,2017-08-17 09:30:00
End,2018-01-01 05:30:00
Period,546 days 00:00:00
Start Value,100.0
End Value,491.394669
Total Return [%],391.394669
Benchmark Return [%],218.108967
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],40.112995
